## Web Scraping Car Damage Images from a Website

***Reminder:*** *Before scraping any website, make sure to check the website policies and follow the rules on how to scrape websites ethically.*

In [ ]:
!pip install selenium undetected_chromedriver beautifulsoup4 pandas pyarrow Pillow requests selenium-wire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 34.6 MB/s eta 0:00:00
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47048 sha256=9de88e829a5ba7457df12fcea3faba51813ada99a3023fd29122af4d85d22bf6
  Stored in directory: /root/.cache/pip/wheels/5c/b9/03/4b6e38f019d6170e8c25df2e1e362d7bdf9ff4012df2dc85c0
Successfully built undetected_chromedriver


### **Import Libraries**

In [ ]:
# import undetected_chromedriver as uc
import hashlib, io, requests, pandas as pd
import time
import random

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver import ChromeOptions
# from seleniumwire import webdriver

from bs4 import BeautifulSoup
from pathlib import Path
from PIL import Image

### **Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### **Launch the WebDriver to open a target URL**
Configure Chrome Options

In [ ]:
options = ChromeOptions()
options.add_argument("--headless=new")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36")
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver = webdriver.Chrome(options=options)

### **Extracting Page URLs**
Each url/page contains the photos for every listed car. Each page will have around 12 photos for the specific car brand. These contains internal and external photos of the car. We only need the external photos so we will be only be extracting those later on.

**Note**: When you encounter a connection error to the website, just rerun the chrome configuration options cell.

Images are from AutobidMaster Website. URLS should be extracted for front, rear, and side panels. The same process should be repeated for each panel.

In [ ]:
## empty list to store the url
page_urls = []

## the range indicates the number of pages in the website
## let's try 30 pages for now
for i in range(1, 30):
  url = f"https://www.autobidmaster.com/en/search/make-toyota,lexus,suzuki/doc-type-c,s/damage-front+end/?page={i}"
  driver.get(url)
  time.sleep(random.randint(1,30))
  elements = driver.find_elements(By.CLASS_NAME, "_1dGf-ymm")

  for item in elements:
    # print(img)
    page_urls.append(item.get_attribute('href'))

driver.quit()

Repeat the process until you finish extracting all urls in all pages

In [ ]:
## check how many page urls were extracted
len(page_urls)

870

#### Saving Extracted Page URLs

In [ ]:
## save the urls to a csv file just in case you need it later on
df = pd.DataFrame(page_urls)
df.to_csv('scraped_page_urls_front.csv')

In [ ]:
## run this code in case runtime suddenly stops and you lose the list
df = pd.read_csv("scraped_page_urls_front.csv")
df.dropna(inplace=True)

### **Extracting Image URLs**

In this step, we will be extracting the urls of the images per page. Extracting a large number of urls is time-consuming so you can do the extraction by batch (depends on your preference).

**Note**: When you encounter a connection error to the website, just rerun the chrome configuration options cell.

In [ ]:
## empty list to store the image urls
image_urls = []

## the range indicates the number of links we extracted in the prior step
## since running this code is time-consuming, you can do it by batch until you finish all links
for page in page_urls[1:50]:
  url = f"{page}"
  driver.get(url)
  time.sleep(random.randint(1,30))
  image_elements = driver.find_elements(By.CLASS_NAME, "_XTkv-img")

  for img in image_elements:
    # print(img)
    image_urls.append(img.get_attribute('data-src'))

driver.quit()

Repeat the process until you finish extracting all image urls.

In [ ]:
## check how many image urls were extracted
len(image_urls)

667

#### Saving Extracted Image URLs

In [ ]:
## save the urls to a csv file just in case you need it later on
df_image_urls = pd.DataFrame(image_urls)
df_image_urls.to_csv('scraped_image_urls_front.csv')

In [ ]:
## run this code in case runtime suddenly stops and you lose the list
# df_image_urls = pd.read_csv("scraped_image_urls_front.csv")
# df_image_urls.dropna(inplace=True)

### **Cleaning the Image URLs**

In [ ]:
## create a list for the url
image_urls = df_image_urls[0].tolist()

## check how many images we have
len(image_urls)

667

In [ ]:
## remove any URLs that contain the substring "_ful.jpg"
## images with substring "_ful.jpg" does not belong to the same car in the list
image_urls = [x for x in image_urls if "_ful.jpg" not in x]

In [ ]:
## checking the list
image_urls

Upon checking, the 5th url in each car image set is the best image to train the model for front damages. Hence, we are only keeping the 5th item in each set.

In [ ]:
## the target string serves as an indication of a new set of car images
target_string = "/build/spa/images/"
result_links = []
temp_links = []  # Temporary storage

for link in image_urls:
  if target_string in link:
    result_links.append(temp_links[4])  ## Keep the 5th link as this is the front damage in a car
    result_links.append(link)
    temp_links = [] # Reset temporary list
  else:
    temp_links.append(link)

result_links.extend(temp_links)  # Add any remaining links

In [ ]:
## checking resulting list
print("There are a total of", len(result_links), "items in the list")

result_links

In [ ]:
## remove the unnecessary image in the list
temp_url = [x for x in result_links if "/build/spa/images/" not in x]

## replace the thumbnail as a full image
new_urls = [url.replace("_thb", "_ful") for url in temp_url]

In [ ]:
## checking the final list of front car damage
print("There are", len(new_urls), "in the list")

new_urls

### **Save Images to Drive Folder**

In [ ]:
## set folder path
data_path = "/content/drive/My Drive/Colab Notebooks/Data/Scraped Data/car-damage-dataset/data2a/combined/front"

In [ ]:
for img in new_urls:
  ## Store the content from the URL to a variable
  image_content = requests.get(img).content

  ## Create a byte object out of image_content and store it in the variable image_file
  image_file = io.BytesIO(image_content)

  ## Use Pillow to convert the Python object to an RGB image
  image = Image.open(image_file).convert("RGB")

  ## Set a file_path variable that points to your directory.
  ## Create a file based on the sha1 hash of 'image_content'.
  ## Use .hexdigest to convert it into a string.
  file_path = Path(data_path, hashlib.sha1(image_content).hexdigest()[:10] + ".jpeg")
  image.save(file_path, "JPEG", quality=120)

Repeat the same process for every car panel.